### This is the S.T.A.S : Sentinel Trading Automated System 
### This one is just the skeleton of a program that will be far more advanced 
### and far more user friendly

In [1]:
!pip install yfinance           #installazione di librerie nel virtualenv del jupyter notebook
!pip install nbformat>=4.2.0
import nbformat

ERROR: Invalid requirement: '#installazione'


# BALANCE SHEET WITH YFINANCE

In [2]:
#Dati di prezzo, bilancio e dati di azione inserendo il ticker code

import yfinance as yf   
import pandas as pd

stock_name = input("Enter the ticker symbol (the code) of the stock you want to analyze: ")
msft = yf.Ticker(stock_name)
print("Now, a description of the stock and the company...\n")

info=[["Info", msft.info],["historical market data", msft.history(period="max")],["Actions", msft.actions],["Dividends", msft.dividends],["Splits", msft.splits],["Financials", msft.financials],
      ["Quarterly financials", msft.quarterly_financials],["Major holders", msft.major_holders],["Institutional holders", msft.institutional_holders],["Balance sheet", msft.balance_sheet],
      ["Quarterly Balance sheet", msft.quarterly_balance_sheet], ["Cash Flow", msft.cashflow], ["Quarterly cash flow", msft.quarterly_cashflow], ["Earnings", msft.earnings],
      ["Quarterly earnings", msft.quarterly_earnings], ["Sustainability", msft.sustainability],  ["Recommendations", msft.recommendations], ["Calendar of next events", msft.calendar],
      ["Options expirations", msft.options]]

for i in range(0,19):            #questo ciclo for scrive il nome del dataset e il dataset
    print(info[i][0])            #tipo "info" seguito dal dataset msft.info, "historical market data" seguito da msft.history(period="max")
    dataset=pd.DataFrame(info[i][1])
    print(dataset)
    print("\n")

Now, a description of the stock and the company...

Info
Empty DataFrame
Columns: [zip, sector, fullTimeEmployees, longBusinessSummary, city, phone, state, country, companyOfficers, website, maxAge, address1, industry, ebitdaMargins, profitMargins, grossMargins, operatingCashflow, revenueGrowth, operatingMargins, ebitda, targetLowPrice, recommendationKey, grossProfits, freeCashflow, targetMedianPrice, currentPrice, earningsGrowth, currentRatio, returnOnAssets, numberOfAnalystOpinions, targetMeanPrice, debtToEquity, returnOnEquity, targetHighPrice, totalCash, totalDebt, totalRevenue, totalCashPerShare, financialCurrency, revenuePerShare, quickRatio, recommendationMean, exchange, shortName, longName, exchangeTimezoneName, exchangeTimezoneShortName, isEsgPopulated, gmtOffSetMilliseconds, quoteType, symbol, messageBoardId, market, annualHoldingsTurnover, enterpriseToRevenue, beta3Year, enterpriseToEbitda, 52WeekChange, morningStarRiskRating, forwardEps, revenueQuarterlyGrowth, sharesOutsta

# STOCK PRICE PLOT

In [3]:
import plotly.express as px

stock_price=pd.DataFrame(info[1][1])
stock_price.reset_index(inplace=True)

closing_price=stock_price["Close"]   #stock_price[close] viene salvato in closing_price perchè servirà spesso più avanti!!
Date=stock_price["Date"]

stockplot = px.line(x=Date, y=closing_price, labels=dict(x="Date", y="Price"))
stockplot.show()

# MOBILE AVERAGES PLOT

In [4]:
def MobileAverage(MArange, Date):   #Creazione della media mobile , cerca la formula su wikipedia della media mobile per capirla
        while (MArange >= len(Date)):
            print("the range is too wide, we advice using no more than 250 days...")
            MArange
    
        MA=[]
        for i in range(0,len(Date)):
            if((len(Date)-(i+MArange)) <= 0):
                    value=closing_price[i:len(Date)].mean()
                    MA.append(value)
            else:
                    value=closing_price[i:i+MArange].mean()
                    MA.append(value)
            
        return MA
    
def MA_signal_Fermat():                  #segnale per capire se la media mobile si flette cambiando da crescente a
    Presignal_List = []                  #decrescente o viceversa usando il teorema di Fermat
    MA10 = MobileAverage(10, Date)
    for i in range(0,len(MA10)-1):
        for k in range(i,len(MA10)-1):
                if((MA10[i+1] - MA10[i]) > 0 and (MA10[k+1] - MA10[k]) < 0):    
                    PreSignal = 'SELL'
                    Presignal_List.append(PreSignal)
                    break
                elif((MA10[i+1] - MA10[i]) < 0 and (MA10[k+1] - MA10[k]) > 0):
                    PreSignal = 'BUY'
                    Presignal_List.append(PreSignal)
                    break
                else:
                    PreSignal = 'NO SIGNAL'
                    Presignal_List.append(PreSignal)
                    break
                    
    return Presignal_List

def Triple_MA():                             #riceve 3 interi e restituisce tre medie mobili
    NumOfMA=input("Number of MA: ")          #una per ogni periodo, il periodo è definito dagli interi
    NumOfMA = int(NumOfMA)
    MArangeList = []
    MA_list = []

    for i in range(0, NumOfMA):    
        MArange = input("Insert the time period: ")
        MArange = int(MArange)
        MArangeList.append(MArange)
        MA = MobileAverage(MArangeList[i] , Date)
        MA_list.append(MA)
        
    return MA_list


def Triple_Crossover_Signal():               #l'intersezioni della media mobile breve con quella media da un segnale buy o sell
    NumOfMA=3                                #in base al fatto che la media breve arrivi da su o da giù
    MArangeValues=[4,9,18]                   #La media mobile breve tocca la lunga e da un altro segnale
    MArangeList = []                         #la media mobile media tocca la lunga e crea un segnale buy o sell
    MA_list = []                             #3 segnali sell danno un sell
    Signal_matrix = []                       #3 segnali buy danno un buy
    Signal_list = []                         #negli altri casi, nessun segnale
    n_buys = 0
    n_sells = 0
    Final_signal_list = []
    
    for i in range(0, NumOfMA):    
        MArange = MArangeValues[i]
        MArangeList.append(MArange)
        MA = MobileAverage(MArangeList[i] , Date)
        MA_list.append(MA)
    
    for i in range(1,len(MA_list[0])):
        for j in range(0,NumOfMA-1):
            if((MA_list[j][i-1] > MA_list[j+1][i-1]) and (MA_list[j][i] == MA_list[j+1][i])):
                Presignal = 'SELL'
            elif(j==0 and (MA_list[j][i-1] > MA_list[j+2][i-1]) and (MA_list[j][i] == MA_list[j+2][i])):
                Presignal = 'SELL'
            elif(j==0 and (MA_list[j][i-1] < MA_list[j+2][i-1]) and (MA_list[j][i] == MA_list[j+2][i])):
                Presignal = 'BUY'
            elif((MA_list[j][i-1] < MA_list[j+1][i-1]) and (MA_list[j][i] == MA_list[j+1][i])):
                Presignal = 'BUY'
            else:
                Presignal = 'NO SIGNAL'
            Signal_list.append(Presignal)
        Signal_matrix.append(Signal_list)
        Signal_list = []
    
    for v in Signal_matrix:
        for n in Signal_list:
            if(Signal_list[n] == 'BUY'):
                ++n_buys
            elif(Signal_list[n] == 'SELL'):
                ++n_sells
            else:
                continue
        if(n_buys == 3 ):
            Final_signal = 'BUY'
        elif(n_sells == 3):
            Final_signal = 'SELL'
        else:
            Final_signal = 'NO SIGNAL'
        Final_signal_list.append(Final_signal)
        
    return Final_signal_list


def Simple_Crossover_Signal():                           #Questo segnale è migliore del triple crossover signal
    NumOfMA=2                                            #e' una intersezione semplice tra due medie mobili e funziona meglio del  
    MArangeValues=[10,50]                                #triplo crossover, anche se questo funziona meglio con i future su commodities
    MArangeList = []
    MA_list = []
    Signal_list = []
    n_buys = 0
    n_sells = 0

    for i in range(0, NumOfMA):    
        MArange = MArangeValues[i]
        MArangeList.append(MArange)
        MA = MobileAverage(MArangeList[i] , Date)
        MA_list.append(MA)
    
    for i in range(1,len(MA_list[0])):
            j=0
            if((MA_list[j][i-1] > MA_list[j+1][i-1]) and (MA_list[j][i] == MA_list[j+1][i])):
                Presignal = 'SELL'
            elif((MA_list[j][i-1] < MA_list[j+1][i-1]) and (MA_list[j][i] == MA_list[j+1][i])):
                Presignal = 'BUY'
            else:
                Presignal = 'NO SIGNAL'
            Signal_list.append(Presignal)
        
    return Signal_list                    

In [5]:
MA70=MobileAverage(70, Date)
print(MA70)

[4.018285693441118, 4.008399980408805, 3.9986856971468243, 3.9937714065824235, 3.996742834363665, 4.009399976049151, 4.023542833328247, 4.032342829023089, 4.040428543090821, 4.048999973705837, 4.056171403612409, 4.058771405901227, 4.061142836298261, 4.0617428404944285, 4.060171410015651, 4.062171408108303, 4.064942833355495, 4.067342836516244, 4.067685696056911, 4.068542838096619, 4.07202855178288, 4.083971408435277, 4.095657120432173, 4.110057118960789, 4.120657120432172, 4.141828554017203, 4.161199981825693, 4.18802855014801, 4.220057119641985, 4.248828547341483, 4.278714258330209, 4.312971404620579, 4.351228547096253, 4.3943142584392, 4.439428544044494, 4.486057114601135, 4.5333428314753945, 4.577742835453578, 4.624114264760698, 4.664742834227425, 4.702314261027745, 4.735428544453212, 4.765599969455174, 4.799485680035183, 4.835199965749468, 4.871114250591823, 4.90274281842368, 4.929857107571193, 4.951228533472333, 4.977114251681736, 5.005428542409624, 5.035857108661107, 5.0687999691

In [6]:
def MAplot():
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    fig = make_subplots(rows=1, cols=1)
    NumOfMA=input("Number of MA: ")
    NumOfMA = int(NumOfMA)
    MArangeList = []
    
    for i in range(0, NumOfMA) :
        MArange = input("Insert the time period: ")
        MArange = int(MArange)
        MArangeList.append(MArange)
        MA = MobileAverage(MArangeList[i] , Date)
        title = "Mobile Av. " + str(MArangeList[i]) + " days"
        fig.add_trace(go.Scatter(name= title, x=Date, y=MA), row=1, col=1)
        
    fig.add_trace(go.Scatter(name="Stock Price", x=Date, y=closing_price), row=1, col=1)
    
    return fig

In [7]:
MAplot()

# BOLLINGER BANDS

In [8]:
def Bollinger():                              #crea le bande di bollinger
    import math
    BigDS = []
    MiniDS = []
    SqrdErr=0
    BollingerMA=MobileAverage(20,Date)
    for i in range(0,len(closing_price)):
        SqrdErr = SqrdErr + ((closing_price[i] - closing_price.mean())**2)     #errore quadrato
        if(i==0):
            continue
        else:
            DS = math.sqrt((SqrdErr)/len(closing_price[0:i]))                   #deviazione standard
        BigDS.append(BollingerMA[i] + (0.6*DS) )
        MiniDS.append(BollingerMA[i] - (0.6*DS) )     #fattore di attualizzazione 0.6 che moltiplica la deviazione standard
    DS_Matrix = [BigDS,BollingerMA,MiniDS]            #di solito il fattore è = a 3, ma con 0.6 sembra funzionare meglio
    
    return DS_Matrix

In [9]:
def Bollinger_Signal():                                       #quando il prezzo tocca la banda alta si vende
    Signal_list = []                                          #quando tocca la banda bassa si compra
    Bollinger_output = Bollinger()
    for i in range(0, len(closing_price)-1):
        if(closing_price[i] == Bollinger_output[0][i]):
            Final_signal = 'SELL'
        elif(closing_price[i] == Bollinger_output[2][i]):
            Final_signal = 'BUY'
        else:
            Final_signal = 'NO SIGNAL'
        Signal_list.append(Final_signal)
    
    return Signal_list

In [10]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go


def Bollinger_plot():
    
    fig = make_subplots(rows=1, cols=1)
    
    DS_Matrix = Bollinger()
    
    fig.add_trace(go.Scatter(name="Stock Price", x=Date, y=closing_price), row=1, col=1)
    fig.add_trace(go.Scatter(name="Big Bollinger Band", x=Date, y=DS_Matrix[0]), row=1, col=1)
    fig.add_trace(go.Scatter(name="Bollinger 20 days mobile average", x=Date, y=DS_Matrix[1]), row=1, col=1)
    fig.add_trace(go.Scatter(name="Little Bollinger Band", x=Date, y=DS_Matrix[2]), row=1, col=1)
    
    return fig

Bollinger_plot()

# RSI 

In [11]:
Date

0      2010-06-29
1      2010-06-30
2      2010-07-01
3      2010-07-02
4      2010-07-06
          ...    
2851   2021-10-25
2852   2021-10-26
2853   2021-10-27
2854   2021-10-28
2855   2021-10-29
Name: Date, Length: 2856, dtype: datetime64[ns]

In [12]:
def RSI(Date, flag=True):                               #cerca su wikipedia la formula del Relative Strenght Index
    import pandas as pd                                 
    
    if(flag):
        timeframe_int = 14
        timeframe = float(timeframe_int)
    else:
        timeframe_int = input("Choose the time period for the RSI (We advice to use 14 days): ")
        timeframe = float(timeframe_int)
    
    Bigdiff_list = []
    Lildiff_list = []
    RSI_list = []
    
    for i in range(0,len(closing_price)):
        if (i==0):
            closing_price_diff_major = 0
            closing_price_diff_major = 0
        else:
            if (closing_price[i] > closing_price[i-1]):
                closing_price_diff_major = closing_price[i] - closing_price[i-1]
                Bigdiff_list.append(closing_price_diff_major)
                Lildiff_list.append(0)
            elif (closing_price[i] < closing_price[i-1]):
                closing_price_diff_minor = closing_price[i-1] - closing_price[i] 
                Lildiff_list.append(closing_price_diff_minor)
                Bigdiff_list.append(0)
            elif (closing_price[i] == closing_price[i-1]):
                Lildiff_list.append(0)
                Bigdiff_list.append(0)
            
            if (i-timeframe < 0):
                Major_Average = sum(Bigdiff_list[0:i])/(timeframe)
                Minor_Average = sum(Lildiff_list[0:i])/(timeframe)
            else:
                Major_Average = sum(Bigdiff_list[(i-timeframe_int):i])/(timeframe)
                Minor_Average = sum(Lildiff_list[(i-timeframe_int):i])/(timeframe)
                
            if (Minor_Average == 0 and Major_Average > 0):
                RSI = 100
                RSI_list.append(RSI)
            elif (Minor_Average == 0 and Major_Average == 0):
                pass
            else:
                RS = Major_Average/Minor_Average
                RSI = 100 - (100/(1+RS))
                RSI_list.append(RSI)
                
    RSI_df=pd.DataFrame(RSI_list)
    RSI_df.rename(columns={0:'RSI'}, inplace= True)
    RSI_date_df = pd.concat([Date, RSI_df], axis=1, join="inner")
    
    return RSI_date_df

In [13]:
def RSI_Signal():
    Signal_List = []
    RSI_df = RSI(Date, flag=True)
    RSI_value = RSI_df['RSI']
    for i in range(0,len(RSI_value)):
        if((RSI_value[i] > 65) and (RSI_value[i] <= 100)):
            Final_Signal = 'SELL'
        elif((RSI_value[i] <= 65) and (RSI_value[i] >= 35)):
            Final_Signal = 'NO SIGNAL'
        elif((RSI_value[i] < 35) and (RSI_value[i] >= 0)):
            Final_Signal = 'BUY'
        Signal_List.append(Final_Signal)
    
    return Signal_List 
        

In [14]:
RSI_df=RSI(Date)
RSI_df

,Date,RSI
0,2010-06-29,0.000000
1,2010-06-30,0.000000
2,2010-07-01,0.000000
3,2010-07-02,0.000000
4,2010-07-06,0.000000
...,...,...
2850,2021-10-22,94.871242
2851,2021-10-25,92.624625
2852,2021-10-26,92.846381
2853,2021-10-27,96.118207


In [15]:
def RSI_Bar_plot():
    RSI_df = RSI(Date)
    
    layout = go.Layout(title='RSI and Stock price',
                   yaxis=dict(title='Stock Price'),
                   yaxis2=dict(title='RSI',
                               overlaying='y',
                               side='right'))
    
    trace1 = go.Scatter(x=RSI_df["Date"], y=closing_price, name="Stock Price", yaxis='y1')
    trace2 = go.Bar(x=RSI_df["Date"], y=RSI_df["RSI"], name="RSI", marker_color='red', yaxis='y2')
    
    data = [trace1, trace2]
    
    return go.Figure(data= data, layout=layout)

In [16]:
def RSI_Line_plot():
    RSI_df = RSI(Date)
    
    layout = go.Layout(title='RSI and Stock price',
                       yaxis=dict(title='Stock Price'),
                       yaxis2=dict(title='RSI',
                                   overlaying='y',
                                   side='right'))
    
    trace1 = go.Scatter(x=RSI_df["Date"], y=closing_price, name="Stock Price", yaxis='y1')
    trace2 = go.Scatter(x=RSI_df["Date"], y=RSI_df["RSI"], name="RSI", yaxis='y2')
    
    data= [trace1, trace2]
    
    return go.Figure(data=data, layout=layout)

In [17]:
RSI_Bar_plot()

In [18]:
RSI_Line_plot()

# CHANGE MOMENTUM OSCILLATOR

In [19]:
def CMO(Date):                                                  #una semplice differenza del prezzo con il prezzo[i] 
    time_frame= input("Insert the time period: ")               #con il prezzo[i-periodo di tempo fisso scelto]
    time_frame = int(time_frame)                                #esempio: prezzo[giorno 30] - prezzo[giorno 10] = CMO[30]
    M_list = []
    
    for i in range(0, len(closing_price)):
        if(i-time_frame<0):
            M = closing_price[i] - closing_price[0]
            M_list.append(M)
        elif(i-time_frame>=0):
            M = closing_price[i] - closing_price[i-time_frame]
            M_list.append(M)
    
    M_df=pd.DataFrame(M_list)
    M_df.rename(columns={0:'CMO'}, inplace= True)
    M_date_df = pd.concat([Date, M_df], axis=1, join="inner")
    
    return M_date_df

In [20]:
def CMO_Signal():
    time_frame= 20
    M_list = []
    
    for i in range(0, len(closing_price)):
        if(i-time_frame<0):
            M = closing_price[i] - closing_price[0]
            M_list.append(M)
        elif(i-time_frame>=0):
            M = closing_price[i] - closing_price[i-time_frame]
            M_list.append(M)
    
    M_df=pd.DataFrame(M_list)
    M_df.rename(columns={0:'CMO'}, inplace= True)
    M_date_df = pd.concat([Date, M_df], axis=1, join="inner")
    
    CMO_value = M_date_df['CMO']
    Signal_List = []
    
    for i in range(0,len(CMO_value)):
        if((CMO_value[i] > 50) and (CMO_value[i] < 150)):
            Final_Signal = 'SELL'
        elif(CMO_value[i] >= 150):
            Final_Signal = 'SELL'
        elif((CMO_value[i] <= 50) and (CMO_value[i] >= -50)):
            Final_Signal = 'NO SIGNAL'
        elif((CMO_value[i] < -50) and (CMO_value[i] > -150)):
            Final_Signal = 'BUY'
        elif((CMO_value[i] <= -150)):
            Final_Signal = 'BUY'
        Signal_List.append(Final_Signal)
    
    return Signal_List
    

In [21]:
def CMO_plot():
    M_date_df=CMO(Date)
    
    layout = go.Layout(title='CMO and Stock price',
                       yaxis=dict(title='Stock Price'),
                       yaxis2=dict(title='CMO',
                                   overlaying='y',
                                   side='right'))
    
    trace1= go.Scatter(name="Stock Price", x=Date, y=closing_price, yaxis='y1')
    trace2= go.Scatter(name="Change Momentum Oscillator", x=M_date_df['Date'], y=M_date_df['CMO'], yaxis='y2')
    
    data= [trace1, trace2]
    
    return go.Figure(data=data, layout=layout)

CMO_plot()

# ELLIOT WAVES

In [22]:
#Creare un riconoscitore delle fasi "5-3" della teoria di elliot, fare prima l'indicatore di Fibonacci

# FIBONACCI INDICATOR

In [23]:
#Creare l'indicatore di fibonacci

# TRADING SIGNAL Algorithm

In [24]:
def Market_Discover_and_Trading_Signal():
    main_list = []
    ticker_list = ['MSFT']
    DEFINITIVE_list = []
    DEFINITIVE_MATRIX = []
    nbuy= 0
    nsell = 0
    nosignal = 0
    
    for stock_name in ticker_list: 
        msft = yf.Ticker(stock_name)
        info=[["Info", msft.info],["historical market data", msft.history(period="max")],["Actions", msft.actions],["Dividends", msft.dividends],["Splits", msft.splits],["Financials", msft.financials],
              ["Quarterly financials", msft.quarterly_financials],["Major holders", msft.major_holders],["Institutional holders", msft.institutional_holders],["Balance sheet", msft.balance_sheet],
              ["Quarterly Balance sheet", msft.quarterly_balance_sheet], ["Cash Flow", msft.cashflow], ["Quarterly cash flow", msft.quarterly_cashflow], ["Earnings", msft.earnings],
              ["Quarterly earnings", msft.quarterly_earnings], ["Sustainability", msft.sustainability],  ["Recommendations", msft.recommendations], ["Calendar of next events", msft.calendar],
              ["Options expirations", msft.options]]
        
        stock_price=pd.DataFrame(info[1][1])
        stock_price.reset_index(inplace=True)

        closing_price=stock_price["Close"]
        Date=stock_price["Date"]
        
        indicator0 = MA_signal_Fermat()
        indicator1 = Simple_Crossover_Signal()
        indicator2 = Triple_Crossover_Signal()
        indicator3 = Bollinger_Signal()
        indicator4 = RSI_Signal()
        indicator5 = CMO_Signal()
        
        sub_list = [indicator0, indicator1, indicator2, indicator3, indicator4, indicator5]
        main_list.append(sub_list)
    
    for j in range(0,len(ticker_list)):
        for n in range(0,len(closing_price)-1):
            for i in range(0,6):
                if(main_list[j][i][n] == 'BUY'):
                    ++nbuy
                elif(main_list[j][i][n] == 'SELL'):
                    ++nsell
                else:
                    ++nosignal
            if(nbuy==6):
                DEFINITIVE = 'BUY'
                OPERATION_AT_PRICE = closing_price[n]
            elif(nsell==6):
                DEFINITIVE = 'SELL'
                OPERATION_AT_PRICE = closing_price[n]
            else:
                DEFINITIVE = 'WAIT'
                OPERATION_AT_PRICE = closing_price[n]
            DEFINITIVE_list.append([DEFINITIVE,OPERATION_AT_PRICE])
        DEFINITIVE_MATRIX.append([ticker_list[j],DEFINITIVE_list])
        DEFINITIVE_list = []
        
    return DEFINITIVE_MATRIX      